In [1]:
import numpy as np
import glob
import math
from PIL import Image
import imageio

In [10]:
def GaussianRange(r):
    GaussianRange = np.zeros(256)
    part = (1 / math.sqrt(2*math.pi* (r**2)))
    for i in range(256):
        GaussianRange[i] = part * np.exp(-(i**2)/(2 * (r**2)))  
    return GaussianRange

In [11]:
def GaussianSpatial(d):
    part = (1 / math.sqrt(2*math.pi* (d**2)))
    GaussianSpatial = np.zeros((2 * d + 1,2 * d + 1))
    range_d = -d + np.arange(2 * d+ 1)
    for i in range_d:
        for j in range_d:
            GaussianSpatial[i+d][j+d]= part * np.exp(-(i **2 + j **2) /(2 * (d**2) ) )
    return GaussianSpatial

In [12]:
def jointBilateralFilter(no_flash, flash, d, r):
    x, y, rgb = no_flash.shape
    noflashImg =np.pad(no_flash, ((d, d), (d, d), (0, 0)), 'symmetric') 
    flashImg = np.pad(flash, ((d, d), (d, d), (0, 0)), 'symmetric')
    outputImg = np.zeros((x,y,rgb), np.uint8)
    GaussianR = GaussianRange(r)
    GaussianS = GaussianSpatial(d)
    
    for i in range(d, d + x):
        for j in range(d, d + y):
            kp = 0
            sum1 = 0
            for k in range (rgb):
                pInten = flashImg[i][j][k]
                qInten = flashImg[i-d : i+d+1 , j-d : j+d+1, k] 
                Rangepad = GaussianR[abs(qInten - pInten)]
                Rangepad2 = Rangepad * noflashImg[i-d : i+d+1 , j-d : j+d+1, k] 
                mul=Rangepad*GaussianS
                kp= np.sum(mul)
                mul2= Rangepad2*GaussianS
                sum1 =  np.sum(mul2)
                outputImg [i-d][j-d][k]= (1/kp)*sum1
                
    return outputImg

def DetailTransfer(flash, e, F_base, denoiseImg):
    h, w, rgb = flash.shape
    F_detail = np.zeros((h,w,rgb), np.uint8)
    for x in range(h):
        for y in range(w):
            for i in range (rgb): #iterate through the image's height, width and channel
                F_detail[x][y][i] = (flash[x][y][i] + e)/(F_base[x][y][i] + e) * denoiseImg[x][y][i]
                
  #  F_detail = np.multiply(F_detail,denoiseImg)
    return F_detail
                         

In [18]:
no_flash = np.array(Image.open('cse273_project/test3a.jpeg'))
flash = np.array(Image.open('cse273_project/test3b.jpeg'))
spatialKern = 3
rangeKern =0.255
filteredimg = jointBilateralFilter(no_flash, flash, spatialKern, rangeKern)
imageio.imwrite('denoise.jpeg',filteredimg)


In [17]:
F_base = jointBilateralFilter(flash,flash,spatialKern,75)
e = 0.02
detailimg = DetailTransfer(flash, e, F_base, filteredimg)
imageio.imwrite('detailimg.jpeg',detailimg)